
** Análise Geral de Compras **

* Número total de consumidores
* Preço médio de compra
* Número total de compras
* Rendimento total


** Informações Demográficas Por Gênero **

* Porcentagem e contagem de compradores masculinos
* Porcentagem e contagem de compradores do sexo feminino
* Porcentagem e contagem de outros / não divulgados


** Análise de Compras Por Gênero **

* Número de compras
* Preço médio de compra
* Valor Total de Compra
* Compras for faixa etária


** Identifique os 5 principais compradores pelo valor total de compra e, em seguida, liste (em uma tabela): **

* Login
* Número de compras
* Preço médio de compra
* Valor Total de Compra
* Itens mais populares


** Identifique os 5 itens mais populares por contagem de compras e, em seguida, liste (em uma tabela): **

* ID do item
* Nome do item
* Número de compras
* Preço do item
* Valor Total de Compra
* Itens mais lucrativos


** Identifique os 5 itens mais lucrativos pelo valor total de compra e, em seguida, liste (em uma tabela): **

* ID do item
* Nome do item
* Número de compras
* Preço do item
* Valor Total de Compra

In [ ]:
# Importação dos pacotes python
import pandas as pd
import matplotlib.pyplot as mlp
import numpy as np

In [ ]:
# Carregando o arquivo
arquivo = "dados_compras.json"
base = pd.read_json(arquivo, orient = "records")
base.head()# retorna as 5 primeiras linhas da base de dados

## Análise Geral de Compras

In [ ]:
#localizar se nos dados há mais de uma venda por consumidor
dados_demo = base.loc[:,["Login","Sexo","Valor"]] #retornar todas as linhas das colunas selecionadas
dados_demo = dados_demo.drop_duplicates("Login")#retira todas os usuário que tiveram mais de uma compra

In [ ]:
total_usuários = dados_demo["Login"].count()# Total de usuários da base demográfica
quantidade_vendas = base["Valor"].count()# Quantidade de vendas realizadas
media_vendas = base["Valor"].mean()# Médidas das vendas realizadas
total_vendas = base["Valor"].sum()# Somatório das vendas realizadas

#criar um dataframe para juntar todas as variáveis criadas 
vendas = pd.DataFrame({"Numero Total de Usuários":total_usuários,
                      "Quantidade de Vendas":quantidade_vendas,
                      "Valor Médio de Vendas": [media_vendas],# valor escalar entre []
                      "Total de vendas":total_vendas},
                       index= ["Total"])

#ajustar formato dos numeros com função map
vendas["Numero Total de Usuários"] = vendas["Numero Total de Usuários"].map("{:,}".format)
vendas["Quantidade de Vendas"] = vendas["Quantidade de Vendas"].map("{:}".format)
vendas["Valor Médio de Vendas"] = vendas["Valor Médio de Vendas"].map("R$ {:,.2f}".format)
vendas["Total de vendas"] = vendas["Total de vendas"].map("R$ {:,.2f}".format)

display(vendas)#display apresenta os dados em formato de tabela
#todo - Alinhar texto do dataframe

## Análise Demográfica por Gênero

In [ ]:
#Localizar todas as linhas das colunas sexo e valor agrupa-las por sexo
vendas_sexo = pd.DataFrame(base.loc[:,["Sexo","Valor"]].groupby("Sexo").count())
#realizar a porcentagem da variável anterior
vendas_sexo_por = pd.DataFrame(base.loc[:,["Sexo","Valor"]].groupby("Sexo").count()/total_vendas*100)
#Realizar a média das vendas por gênero
vendas_sexo = base.loc[:,["Sexo","Valor"]].groupby("Sexo").mean()

analise = pd.concat([vendas_sexo,vendas_sexo_por], axis= 0, ignore_index= True)

display(analise)
#todo como colocar dois group by em um dataframe
#Todo ajuste de formatação dos valores

In [ ]:
# Criar dois dataframes exemplo
df1 = pd.DataFrame({'coluna1': ['A', 'B', 'C'], 'valor1': [1, 2, 3]})
df2 = pd.DataFrame({'coluna1': ['D', 'E', 'F'], 'valor2': [4, 5, 6]})

# Concatenar os dois dataframes verticalmente
result = pd.concat([df1, df2], axis=0, ignore_index=True)
#result = pd.merge(df1, df2, on='coluna1')

display(result)

In [ ]:
#contagem de compradores
total_masc = len(base.loc[base["Sexo"] == "Masculino"])
total_fem = len(base.loc[base["Sexo"] == "Feminino"])
total_outro = len(base.loc[base["Sexo"] == "Outro / Não Divulgado"])

#% de cada gênero
por_masc = (total_masc / (base["Sexo"].count()))*100
por_fem = (total_fem / (base["Sexo"].count()))*100
por_outros = (total_outro / (base["Sexo"].count()))*100

#Dataframe
sexo = pd.DataFrame({ 
                    "Total":[total_masc,total_fem,total_outro],
                    "%":[por_masc,por_fem,por_outros]},
                    index= ["Masculino","Feminino", "Outros / Não Divulgado"])

#Formatação de Números usando a função map reduce
sexo["%"] = sexo["%"].map("{:,.2f}".format)
#Todo: alinhar texto do Dataframe

#Display
display(sexo)

In [ ]:
#Tipo de plot
fig, ax = mlp.subplots()

#dados do plot
gen = ['Masculino','Feminino','Outro / Não Divulgado']
counts = sexo["Total"]
bar_colors = ['tab:red', 'tab:blue','tab:orange']

#eixo x
ax.bar(gen, counts, color=bar_colors)

#eixo y
ax.set_ylabel('Qntd')

#título
ax.set_title('Qntd por Gênero')

#show
mlp.show()

In [ ]:
#plot das porcentagens

labels = 'Masculino', 'Feminino', 'Outros / Não Divulgado'
sizes = sexo["Total"]
explode = (0.2, 0.1, 0.1)  

fig1, ax1 = mlp.subplots()
ax1.pie(sizes, explode=explode, labels=labels, autopct='%1.1f%%',
        shadow=True, startangle=90)
ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

mlp.show()

## Análise de Compras Por Gênero

In [ ]:
# Análise de compras por gênero
base_gen = base.loc[:,["Sexo","Valor"]]
display(base_gen)

In [ ]:
#agrupar valores por genero com o groupyby
total_gen = base_gen.groupby("Sexo").sum()
total_me = base_gen.groupby("Sexo").mean()
total_ = base_gen.groupby("Sexo").sum()/ total_vendas*100

display(total_me)

## Consumidores Mais Populares (Top 5)

In [ ]:
# Implemente aqui sua solução

## Itens Mais Populares

In [ ]:
# Implemente aqui sua solução

## Itens Mais Lucrativos

In [ ]:
# Implemente aqui sua solução